In [26]:
import numpy as np
import csv
from itertools import permutations, cycle
import time
import random

file = open('/home/johannes/Work/INF3490/oblig1/european_cities.csv', 'r+')
reader = csv.reader(file,delimiter=";")

distance = np.zeros((24,24))
cities = []

i = 0
for row in reader:
    #print row
    if i == 0:
        for j in range(len(row)):
            cities.append(row[j])
    else:
        for j in range(len(distance)):
            distance[i-1,j] = float(row[j])
    i += 1


In [27]:
# Exhaustive search

start_time = time.time()

N = 6
A = distance[0:N,0:N]

a = np.arange(N)

def dist(cities):
    tour = np.zeros(len(cities))
    for i in range(len(cities)-1):
        tour[i] = A[cities[i],cities[i+1]]
    tour[-1] = A[cities[-1], cities[0]]
    return np.sum(tour)
    
    
best_sequence = np.zeros(len(cities))
best = 100000000
for i in permutations(a):
    temp = dist(i)
    if temp < best:
        best = temp
        best_sequence = i
        
elapsed_time = time.time() - start_time
        
print best
print best_sequence
print ("Time elapsed is %f seconds" %(elapsed_time))


def index_to_city_name(city_names, city_indices):
    best_cities = []
    for i in range(len(city_indices)):
        best_cities.append(city_names[int(city_indices[i])])
    return best_cities

print index_to_city_name(cities, best_sequence)

5018.81
(0, 1, 4, 5, 2, 3)
Time elapsed is 0.007218 seconds
['Barcelona', 'Belgrade', 'Bucharest', 'Budapest', 'Berlin', 'Brussels']


In [28]:
# Hill climbing

def hillclimb(cities):
    init = cities
    np.random.shuffle(init)
    
    best = dist(init)
    count = 0
    best_sequence = init.copy()
    while count < 10:        
        number1 = np.random.randint(N)
        number2 = np.random.randint(N)
        
        while number1 == number2:
            number2 = np.random.randint(N)
        
        value1 = init[number1]
        value2 = init[number2]
        
        init[number1] = value2
        init[number2] = value1
        
        temp = dist(init) 
        
        if temp < best:
            best = temp
            best_sequence = init.copy()
            count = 0
        else:
            count += 1
        
    return best, best_sequence

best, best_sequence = hillclimb(a)
print best, best_sequence  
print index_to_city_name(cities, best_sequence)

# WTF happens with the zeros??
    

5270.45 [1 4 2 3 0 5]
['Belgrade', 'Bucharest', 'Berlin', 'Brussels', 'Barcelona', 'Budapest']


In [114]:
# Genetic algorithm

# ----------------------------------------------------------------------------------------------

# Create population
def mk_population(size,cities):
    population = np.zeros((size,len(cities)))
    for i in range(size):
        np.random.shuffle(cities) 
        population[i] = cities
    return population

population = mk_population(6,a)
#print "population is ", "\n", v
#print "\n"

# ----------------------------------------------------------------------------------------------


# Create tournament group
def mk_tournament(population,size):
    
    tournament_size = population.shape[0]/size
    if (float(population.shape[0])/size).is_integer()!=True:
        print "Change population or tournament size!"
    #print tournament_size
        
    tournaments = np.zeros((size,tournament_size))
    population_i = np.arange(population.shape[0])
    np.random.shuffle(population_i)
    
    #print population_i
    count = 0
    for i in range(size):
        for j in range(tournament_size):
            tournaments[i,j] = population_i[count]
            count += 1
            
    
    return tournaments


tournaments = mk_tournament(v,2)    
#print "tournaments is ", "\n", w
#print "\n"

# ----------------------------------------------------------------------------------------------


# create parents
# takes in the tournaments matrix from mk_tournament, the population, and which tournament you want
def mk_parents(tournaments,population,tournament):
    
    # create the vectors with city indices of the whole tour
    actual_tournament = np.zeros((tournaments.shape[1],population.shape[1]))
    for i in range(tournaments.shape[1]):   # should it be tournament?
        actual_tournament[i] = population[int(tournaments[tournament,i])] 
    #print "tournament tour is", "\n", actual_tournament, "\n"
    
    
    # Find the parents
    distances = np.zeros(len(actual_tournament))
    for i in range(len(actual_tournament)):
        distances[i] = dist(actual_tournament[i])
    
    parents = np.argpartition(distances,-1)[:2]
    #print "parents is ", "\n", parents, "\n"
    #print "distances is ", "\n", distances, "\n"
    
    global_coord_parents = np.zeros(len(parents))
    for i in range(len(global_coord_parents)):
        global_coord_parents[i] = tournaments[tournament, parents[i]]
    #print "actual coord is ", "\n", global_coord_parents, "\n"
    
    outcast = np.argpartition(distances,-1)[-1]
    #print "Worst value is", "\n", outcast, "\n"
    global_coord_outcast = tournaments[tournament,outcast]
    #print "The actual coord. for the outcast is \n", global_coord_outcast, "\n"

    
    return global_coord_parents, global_coord_outcast

    
parents_coord,outcast_coord = mk_parents(w,v,0)

# ----------------------------------------------------------------------------------------------
# PMX (Partially mapped crossover)

def crossover(population, parents):
    parent1 = np.zeros(population.shape[1])
    parent2 = np.zeros(population.shape[1])
    parent1 = population[parents[0],:]
    parent2 = population[parents[1],:]
    N = len(parent1)
    child = np.zeros(N)
    
    print parent1,parent2
    
    find_zero1 = np.where(parent1 == 0.0)
    find_zero2 = np.where(parent2 == 0.0)
    
    parent1[find_zero1[0]] = N+1
    parent2[find_zero2[0]] = N+1
    
     # Creating segment size and start position
    segment_size = np.random.randint(len(parent1))  # The actual size is plus one
    segment_start_pos = np.random.randint(len(parent1)-segment_size) 
    
    # Copy segment from parent1 to child
    segment = np.zeros(segment_size+1)
    segment = parent1[segment_start_pos:segment_start_pos+segment_size+1]
    child[segment_start_pos:segment_start_pos+segment_size+1] = segment
    
    print segment
    
    
    for i in range(len(child)):
        if not parent2[i] in child:
            mv_to = np.where(parent2 == parent2[i])
            while True:
                if child[mv_to[0]] == 0.0:  # have to make this zero
                    child[mv_to[0]] = parent2[i]
                    break
                else:
                    mv_to = np.where(parent2 == child[mv_to[0]])
    
    for i in range(N):
        if child[i] == 0.0:  # have to find a better way for this...
            child[i] = parent2[i]
            
    
    insert_zero = np.where(child == N+1)
    child[insert_zero[0]] = 0.0
    
    return child
                    

# ----------------------------------------------------------------------------------------------


# Mutation

def inversion(child):
    segment_size = np.random.randint(len(child))+1
    start_pos = np.random.randint(len(child)-(segment_size-1))
    end_pos = start_pos + segment_size
    segment = child[start_pos:end_pos]
    
    reverse_segment = segment[::-1]
    
    child[start_pos:end_pos] = reverse_segment
    
    return child
       
def decision(probability):
    return random.random() < probability
    
    
def mutation(child,probability):
    if decision(probability) == True:
        child = inversion(child)
    return child
        
    
# ----------------------------------------------------------------------------------------------
    
# Create new population

def new_population(population, children, outcasts):
    new_pop = np.zeros(population.shape)
    new_pop = population.copy()
    for i in range(len(outcasts)):
        new_pop = np.delete(population, outcasts[i], 0)
    for i in range(len(children)):
        new_pop = np.vstack([new_pop,population[children[i]]])

    return new_pop


/home/johannes/.local/lib/python2.7/site-packages/ipykernel/__main__.py:13: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/johannes/.local/lib/python2.7/site-packages/ipykernel/__main__.py:14: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [115]:
# main program....

NoG = 10 # number of generations
population_size = 20 # size of the population
NoC = 10 # number of cities
city_vec = np.arange(NoC)
TournamentSize = 5
NoT = population_size/TournamentSize # Number of tournaments 
Pr = 0.03 # Probability of mutation
population_distances = np.zeros(population_size)
population_avg = np.zeros(NoG)

population = mk_population(N,city_vec)
for i in range(N):
    tournaments = mk_tournament(population,TournamentSize)
    parents = np.zeros((NoT, 2))
    outcast = np.zeros((NoT, 1))
    child = np.zeros((NoT,1))
    
    for j in range(NoT):
        parents[j], outcast[j] = mk_parents(tournaments,population,j)
        child[j] = crossover(population,parents[j])
        child[j] = mutation(child[j],Pr)
        
    population = new_population(population, child, outcast)
    
    for k in range(population_size):
        population_distances[k] = dist(population[k,:])
        
    population_avg[i] = np.average(population_distances)
        
    

print population_avg

/home/johannes/.local/lib/python2.7/site-packages/ipykernel/__main__.py:13: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


IndexError: index 8 is out of bounds for axis 1 with size 6